In [34]:
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
import openai

# For ChatGPT-4
openai.api_key = os.getenv("OPENAI_API_KEY")
# alternatively set LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = ""  # "true"

try:
    models = openai.models.list()
    print("API key is valid. Available models:", [model.id for model in models.data])
except openai.error.AuthenticationError as e:
    print("API key is invalid:", str(e))

openai_llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=os.getenv("OPENAI_API_KEY"))  # gpt-4-turbo - more correct grammar info


API key is valid. Available models: ['whisper-1', 'dall-e-2', 'gpt-3.5-turbo-16k', 'tts-1-hd-1106', 'tts-1-hd', 'gpt-4-turbo-2024-04-09', 'gpt-4-0125-preview', 'gpt-4-turbo-preview', 'gpt-4-turbo', 'gpt-3.5-turbo-instruct-0914', 'gpt-3.5-turbo', 'gpt-3.5-turbo-instruct', 'text-embedding-3-small', 'tts-1', 'text-embedding-3-large', 'gpt-4-1106-preview', 'babbage-002', 'gpt-3.5-turbo-0125', 'tts-1-1106', 'gpt-4o-2024-05-13', 'dall-e-3', 'gpt-4-0613', 'gpt-4o', 'text-embedding-ada-002', 'gpt-4', 'davinci-002', 'gpt-3.5-turbo-1106']


In [3]:
def llama3_request(prompt):
    # This function should interact with your local Llama3 model.
    # Replace the below code with actual API or command-line interaction.
    import subprocess
    result = subprocess.run(['ollama', 'run', 'llama3', '--prompt', prompt], capture_output=True, text=True)
    return result.stdout


In [65]:
model = ChatOpenAI()
parser = StrOutputParser()

# Prompt templates
translation_system_template = """Given the following text in {source_language}, 
translate the word marked as <b>{word}</b> from {source_language} to {target_language}. 
Give the meaning of it in the exact sentence.
Put into result only word translation without surrounding <b></b>"""

translation_prompt_template = ChatPromptTemplate.from_messages([
    ('system', translation_system_template),
    ('user', '{text}')
])
translation_chain = translation_prompt_template | model | parser


dictionary_system_template = """Given the following text in {source_language}, 
write {source_language} - {target_language} dictionary article for the word marked as <b>{word}</b>.
The article should be in {target_language}.
Include grammar attributes - part of the speech, genre, number, countability and other grammar attributes.
All grammar attributes should be on one line in a compact way like in good dictionaries.
Include different meanings, declination table and other information you expect to see in a good dictionary like Oxford, but do not include examples.
And do not include translation of the text provided.
Give the result in HTML formatting, without any block marks."""

dictionary_prompt_template = ChatPromptTemplate.from_messages([
    ('system', dictionary_system_template),
    ('user', '{text}')
])
dictionary_chain = dictionary_prompt_template | model | parser


examples_system_template = """Given the following text in {source_language}, 
provide at least seven examples in {source_language} of sentences using the word marked as <b>{word}</b> in his meaning in the sentence.
Do not give as an example the usage in the given text.
Give translations of the examples to {target_language}
Each example in a separate paragraph (<p>).
Give the result in HTML formatting, without any block marks.
"""

examples_prompt_template = ChatPromptTemplate.from_messages([
    ('system', examples_system_template),
    ('user', '{text}')
])
examples_chain = examples_prompt_template | model | parser


explanation_system_template = """Given the following text in {source_language}, 
using only {source_language} explain the meaning of the word marked as <b>{word}</b> in exact sentence.
You are a teacher who read the text with your student, use the exact context in the sentence for the explanation.
"""

explanation_prompt_template = ChatPromptTemplate.from_messages([
    ('system', explanation_system_template),
    ('user', '{text}')
])
explanation_chain = explanation_prompt_template | model | parser


# Function to interact with Llama3
def llama3_chain(prompt_template, **kwargs):
    prompt = prompt_template.format(**kwargs)
    response = llama3_request(prompt)
    return response


In [30]:
def get_translation_examples_and_explanation(text: str, word: str, source_language: str, target_language: str, model: str = 'gpt'):
    # Mark the specific word in the text
    marked_text = text.replace(word, f"<b>{word}</b>", 1)
    if model == 'gpt':
        translation = translation_chain.invoke({
            "text": marked_text, "word": word, "source_language": source_language, "target_language": target_language})
        dictionary = dictionary_chain.invoke({
            "text": marked_text, "word": word, "source_language": source_language, "target_language": target_language})
        examples = examples_chain.invoke({
            "text": marked_text, "word": word, "source_language": source_language, "target_language": target_language})
        explanation = explanation_chain.invoke({
            "text": marked_text, "word": word, "source_language": source_language, "target_language": target_language})
    elif model == 'llama3':
        translation = llama3_chain(translation_prompt_template, text=marked_text, word=word, source_language=source_language, target_language=target_language)
        # examples = llama3_chain(example_prompt_template, text=marked_text, word=word, source_language=source_language, target_language=target_language)
        # explanation = llama3_chain(explanation_prompt_template, text=marked_text, word=word, source_language=source_language)
    return f"{translation}<br><br>{dictionary}<br><br>{examples}<br><br>{explanation}"



In [68]:
# Example usage
# serbian_text = "Ljubav je najlepša stvar na svetu. Ljubav pokreće sve."
# word_to_translate = "Ljubav"
serbian_text = "pokreće sve. list sa drveta je pao na zemlju. Na stolu je bio list papira sa važnim beleškama. Ljubav je najlepša "
word_to_translate = "list"
source_language = "Serbian"
target_language = "English"

# Test with ChatGPT-4
gpt = get_translation_examples_and_explanation(serbian_text, word_to_translate, source_language, target_language, model='gpt')

# Test with Llama3
# translation_llama, examples_llama, explanation_llama = get_translation_examples_and_explanation(serbian_text, word_to_translate, source_language, target_language, model='llama3')

from IPython.display import display, HTML

print("ChatGPT-4 Results:")
display(HTML(gpt))

# print("Llama3 Results:")
# display(HTML(html_result_llama))


ChatGPT-4 Results:


Case,Singular,Plural
Nominative,list,listovi
Genitive,lista,listova
Dative,listu,listovima
Accusative,list,listove
Instrumental,listom,listovima
Locative,listu,listovima
Vocative,liste,listovi
